In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import nltk.stem.porter
import numpy as np
#from imblearn.over_sampling import RandomOverSampler
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [11]:
stemmer = nltk.stem.PorterStemmer()
f= open('output.txt', 'w')

In [12]:
data = pd.read_csv("input.csv")
data

,genre,song,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Hip Hop/Rap,Verse One:\n\nAlright I might\nHave had a litt...,NaN,105.0,Hip Hop/Rap
1,Rock,Adam Ant/Marco Pirroni\nEvery girl is a someth...,NaN,533.0,Rock
2,Rock,"I've just erased it's been a while, I've got a...",NaN,79.0,Country
3,R&B,Little darling \nWhere you've been so long \nI...,NaN,40.0,Electronic
4,Rock,"Lead Vocal by Greg\n\nWell, these late night c...",NaN,85.0,Pop
...,...,...,...,...,...
890,Rock,Born from some mother's womb\nJust like any ot...,NaN,NaN,NaN
891,Rock,"This wanting more from me is tearing me, it's ...",NaN,NaN,NaN
892,Pop,"Hello, it's me\nI was wondering if after all t...",NaN,NaN,NaN
893,Pop,"Oooh, oooh, oooh, oooh\n\nShe, she ain't real\...",NaN,NaN,NaN


In [13]:
def stemmed_words(doc):
    return(stemmer.stem(w) for w in analyzer(doc))

In [14]:
train = data.loc[:,["genre", "song"]]
train

,genre,song
0,Hip Hop/Rap,Verse One:\n\nAlright I might\nHave had a litt...
1,Rock,Adam Ant/Marco Pirroni\nEvery girl is a someth...
2,Rock,"I've just erased it's been a while, I've got a..."
3,R&B,Little darling \nWhere you've been so long \nI...
4,Rock,"Lead Vocal by Greg\n\nWell, these late night c..."
...,...,...
890,Rock,Born from some mother's womb\nJust like any ot...
891,Rock,"This wanting more from me is tearing me, it's ..."
892,Pop,"Hello, it's me\nI was wondering if after all t..."
893,Pop,"Oooh, oooh, oooh, oooh\n\nShe, she ain't real\..."


In [17]:
test = pd.read_csv("input.csv")
test = test.loc[:,["genre", "song"]]
test

,genre,song
0,Hip Hop/Rap,Verse One:\n\nAlright I might\nHave had a litt...
1,Rock,Adam Ant/Marco Pirroni\nEvery girl is a someth...
2,Rock,"I've just erased it's been a while, I've got a..."
3,R&B,Little darling \nWhere you've been so long \nI...
4,Rock,"Lead Vocal by Greg\n\nWell, these late night c..."
...,...,...
890,Rock,Born from some mother's womb\nJust like any ot...
891,Rock,"This wanting more from me is tearing me, it's ..."
892,Pop,"Hello, it's me\nI was wondering if after all t..."
893,Pop,"Oooh, oooh, oooh, oooh\n\nShe, she ain't real\..."


In [63]:
import nltk

In [64]:
analyzer = CountVectorizer().build_analyzer()
vectorizer = CountVectorizer(analyzer=stemmed_words,
                             tokenizer=nltk.tokenize.TreebankWordTokenizer(),
                             lowercase=True,
                             preprocessor=None,
                             max_features=5000
                             )

analyzer = CountVectorizer().build_analyzer()
vectorizer = CountVectorizer(analyzer=stemmed_words,
                             tokenizer=nltk.tokenize.TreebankWordTokenizer(),
                             lowercase=True,
                             preprocessor=None,
                             max_features=5000
                             )

In [65]:
train_data_features = vectorizer.fit_transform([r for r in train["song"]])
test_data_features = vectorizer.transform([r for r in test["song"]])
train_data_features

<895x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 82665 stored elements in Compressed Sparse Row format>

In [66]:
train_data_features = train_data_features.toarray()
test_data_features = test_data_features.toarray()

In [67]:
print("fitting for Naive bayes\n")
NB_model = MultinomialNB()
NB_model.fit(train_data_features, train["genre"])
#f.write("\nOutput from Naive Bayes Multi Normal:\n")
NB_predicted = NB_model.predict(test_data_features)
NB_predicted

fitting for Naive bayes



array(['Hip Hop/Rap', 'Rock', 'Rock', 'R&B', 'Rock', 'Rock', 'Rock',
       'Pop', 'Rock', 'Rock', 'Rock', 'Rock', 'Hip Hop/Rap', 'Rock',
       'Rock', 'Electronic', 'Rock', 'Rock', 'Rock', 'Hip Hop/Rap',
       'Rock', 'Rock', 'Pop', 'Country', 'Rock', 'Rock', 'Rock',
       'Hip Hop/Rap', 'Hip Hop/Rap', 'Rock', 'Rock', 'Rock', 'R&B',
       'Rock', 'Pop', 'Hip Hop/Rap', 'Rock', 'Rock', 'Rock', 'Country',
       'Rock', 'Rock', 'Hip Hop/Rap', 'Country', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'R&B', 'Pop', 'Pop', 'Rock', 'Electronic',
       'Hip Hop/Rap', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Hip Hop/Rap', 'Pop', 'Rock', 'Rock', 'Rock', 'Rock',
       'Hip Hop/Rap', 'Rock', 'Rock', 'Rock', 'Rock', 'Country', 'Rock',
       'Rock', 'Rock', 'Pop', 'Rock', 'Rock', 'Hip Hop/Rap', 'Pop',
       'Rock', 'Hip Hop/Rap', 'Rock', 'Rock', 'Rock', 'Pop', 'Country',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Hip Hop/Rap', 'Rock',
       'Rock', 'R&B'

NB_model.score()

In [68]:
print("fitting for SVC\n")
log_model = SVC()
log_model.fit(train_data_features, train["genre"])
#f.write("\nOutput from SVC Normal:\n")
SVC_predicted = log_model.predict(test_data_features)
SVC_predicted

fitting for SVC



array(['Hip Hop/Rap', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Pop', 'Rock', 'Rock', 'Rock', 'Rock', 'Hip Hop/Rap', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Hip Hop/Rap', 'Rock',
       'Rock', 'Pop', 'Rock', 'Rock', 'Rock', 'Rock', 'Hip Hop/Rap',
       'Hip Hop/Rap', 'Rock', 'Rock', 'Rock', 'R&B', 'Rock', 'Rock',
       'Hip Hop/Rap', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Hip Hop/Rap', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'R&B', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock', 'Hip Hop/Rap',
       'Pop', 'Rock', 'Rock', 'Rock', 'Rock', 'Hip Hop/Rap', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Hip Hop/Rap', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Hip Hop/Rap', 'Pop', 'Rock',
       'Hip Hop/Rap', 'Rock', 'Rock', 'Rock', 'Pop', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Rock', 'Hip Hop/Rap', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Ro

In [69]:
print("fitting for LR\n")
log_model = LogisticRegression()
log_model.fit(train_data_features, train["genre"])
#f.write("\nOutput from SVC Normal:\n")
log_predicted = log_model.predict(test_data_features)
log_predicted

fitting for LR



C:\Users\Aymane\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(['Hip Hop/Rap', 'Rock', 'Rock', 'R&B', 'Rock', 'Rock', 'Rock',
       'Pop', 'Rock', 'Rock', 'Country', 'Rock', 'Hip Hop/Rap', 'Rock',
       'Rock', 'Electronic', 'Rock', 'Rock', 'Rock', 'Hip Hop/Rap',
       'Rock', 'Rock', 'Pop', 'Country', 'Rock', 'Rock', 'Rock',
       'Hip Hop/Rap', 'Hip Hop/Rap', 'Country', 'Rock', 'Rock', 'R&B',
       'Rock', 'Pop', 'Hip Hop/Rap', 'Rock', 'Rock', 'Rock', 'Country',
       'Rock', 'R&B', 'Hip Hop/Rap', 'Country', 'Rock', 'Rock',
       'Electronic', 'Rock', 'Rock', 'Rock', 'R&B', 'Pop', 'Pop', 'Rock',
       'Electronic', 'Hip Hop/Rap', 'Rock', 'Rock', 'Rock', 'Rock',
       'Rock', 'Rock', 'Rock', 'Hip Hop/Rap', 'Pop', 'Rock', 'Rock',
       'Rock', 'Rock', 'Hip Hop/Rap', 'Pop', 'Rock', 'R&B', 'Rock',
       'Country', 'Hip Hop/Rap', 'Rock', 'Rock', 'Pop', 'Rock', 'Rock',
       'Hip Hop/Rap', 'Pop', 'Rock', 'Hip Hop/Rap', 'Pop', 'Rock', 'Rock',
       'Pop', 'Country', 'Rock', 'Rock', 'Rock', 'Rock', 'Rock',
       'Hip Hop/Rap', 'Rock'

In [70]:
from sklearn.metrics import classification_report
#print(classification_report(test_data_features, log_predicted))
log_model.score(test_data_features, log_predicted)

1.0

In [ ]:
 mn  